In [1]:
# import the dependencies:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
import json
from pprint import pprint
# Import the API key.
from config import weather_api_key
import numpy as np
import time
from datetime import datetime
import random
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Create a set of random latitude and longitude combination of 2000 + cities:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)
# the zip method combines the lats and lngs into a list of paired coordinates.

In [3]:
# Add the latitudes and longitudes to a list:
coordinates = list(lats_lngs)

In [4]:
# Get the nearest city  using the citipy module:
cities= []

# Identify each city for nearest lat and lng in combination.

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # if the city is unique then add it to our cities list.
    
    if city not in cities:
        cities.append(city)
        
# print the city count to confirm sufficient count.
len(cities)


748

In [5]:
# Perform an API call:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# print(url)

In [6]:
# Create and empty list to hold the weather data and print the beginning of the logging:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
      # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # print(requests.get(city_url).json())
    
       # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": current_description,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,})
        
                        
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | pecanha
Processing Record 2 of Set 1 | port alfred
Processing Record 3 of Set 1 | mackenzie
Processing Record 4 of Set 1 | cherskiy
Processing Record 5 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 6 of Set 1 | palabuhanratu
City not found. Skipping...
Processing Record 7 of Set 1 | tukrah
Processing Record 8 of Set 1 | hermanus
Processing Record 9 of Set 1 | progreso
Processing Record 10 of Set 1 | altamont
Processing Record 11 of Set 1 | east london
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | haines junction
Processing Record 14 of Set 1 | hobart
Processing Record 15 of Set 1 | kearney
Processing Record 16 of Set 1 | punta arenas
Processing Record 17 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 18 of Set 1 | bathsheba
Processing Record 19 of Set 1 | kharod
Processing Record 20 of Set 1 | dikson
Processing Record 21 of Se

In [7]:
len(city_data)

685

In [8]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pecanha,-18.5486,-42.5569,clear sky,55.18,77,3,2.77,BR,2022-07-25 02:45:39
1,Port Alfred,-33.5906,26.8910,broken clouds,57.24,60,65,3.74,ZA,2022-07-25 02:45:39
2,Mackenzie,55.2999,-123.1698,overcast clouds,66.42,63,100,3.44,CA,2022-07-25 02:45:39
3,Cherskiy,68.7500,161.3000,clear sky,64.27,36,9,7.18,RU,2022-07-25 02:43:15
4,Tukrah,32.5341,20.5791,clear sky,77.00,88,0,4.68,LY,2022-07-25 02:45:40


In [9]:
# city_url

In [10]:
new_column_order = ['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed', 'Current Description']
city_data_df = city_data_df[new_column_order]

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Pecanha,BR,-18.5486,-42.5569,55.18,77,3,2.77,clear sky
1,Port Alfred,ZA,-33.5906,26.8910,57.24,60,65,3.74,broken clouds
2,Mackenzie,CA,55.2999,-123.1698,66.42,63,100,3.44,overcast clouds
3,Cherskiy,RU,68.7500,161.3000,64.27,36,9,7.18,clear sky
4,Tukrah,LY,32.5341,20.5791,77.00,88,0,4.68,clear sky


In [11]:
# export the file into a csv file:
output_data_file = "WeatherPy_Database.csv"

city_data_df.to_csv(output_data_file, index_label="City_ID")